# Set up


## Imports

In [1]:
%matplotlib inline

In [2]:
import sys
import os
from pathlib import Path
import math

In [3]:
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_context(context="talk", font_scale=1, rc=None)
sns.set_style('whitegrid')

In [4]:
# Load the "autoreload" extension
%load_ext autoreload
%autoreload 2

In [5]:
import pandas as pd
from pandas import IndexSlice as idx
import numpy as np

from collections import OrderedDict

from munch import Munch, munchify
from tqdm import tqdm

In [6]:
import engarde.decorators as ed
import pyparsing as p

In [7]:
# import crunchers.sklearn_helpers.exploration as expl
# import crunchers.statsmodels_helpers.lazy_stats as stats
# import crunchers.pandas_helpers.transformations as xform

In [28]:
import biorep_etl.data.hreg_load_recode as loading
import biorep_etl.data.field_definitions.hreg_redcap_dump as rcd
from biorep_etl.data.parsers import BranchLogicParser, build_choices_parser

## Constants

### Get info for converting/moving this file to `reports`

In [9]:
# %%javascript
# var kernel = IPython.notebook.kernel;
# var thename = window.document.getElementById("notebook_name").innerHTML;
# var command = "IPYNB_NAME = " + "'"+thename+"'";
# kernel.execute(command);

In [10]:
# rv = Munch()

# rv.rep_id = 'yyy-mm-dd'
# rv.desc = 'blank'
# rv.ipnb = Path('{IPYNB_NAME}.ipynb'.format(IPYNB_NAME=IPYNB_NAME))
# rv.html = '{ipnb}.html'.format(ipnb=rv.ipnb.stem)
# rv.files = '{ipnb}_files'.format(ipnb=rv.ipnb.stem)
# rv.reprt_dir = '../../reports/{IPYNB_NAME}/'.format(IPYNB_NAME=IPYNB_NAME)

### Other Constants

## Paths

In [11]:
data_ = "../data/raw/registry/HarvardInflammatoryB_DATA_2017-01-20_0924.csv"
data_dict_ = "../data/raw/registry/HarvardInflammatoryBowelDiseas_DataDictionary_2017-01-20.csv"

## Functions

## Loading

### Data Dict

In [13]:
cb = loading.load_data_dict(data_dict_=data_dict_).dropna(axis=1, how='all')
cb.head(2)

,Form Name,Section Header,Field Type,Field Label,"Choices, Calculations, OR Slider Labels",Field Note,Text Validation Type OR Show Slider Number,Text Validation Min,Text Validation Max,Identifier?,Branching Logic (Show field only if...),Required Field?,Custom Alignment,Matrix Group Name
Variable / Field Name,,,,,,,,,,,,,,
subid,registration_visit,NaN,text,Subject ID,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
visitdate,registration_visit,Form RV: Registration Visit,text,Date of visit at which patient was consented:,NaN,NaN,date_mdy,NaN,NaN,NaN,NaN,y,NaN,NaN


In [14]:
cb['Field Type'].value_counts()

radio          214
text            80
checkbox        49
dropdown        34
yesno            6
descriptive      5
calc             3
Name: Field Type, dtype: int64

In [15]:
cb[cb['Field Type'] == 'radio'].head()

,Form Name,Section Header,Field Type,Field Label,"Choices, Calculations, OR Slider Labels",Field Note,Text Validation Type OR Show Slider Number,Text Validation Min,Text Validation Max,Identifier?,Branching Logic (Show field only if...),Required Field?,Custom Alignment,Matrix Group Name
Variable / Field Name,,,,,,,,,,,,,,
noconsent,registration_visit,NaN,radio,"If no, provide reason","1, Not interested in Research Studies | 2, Pri...",NaN,NaN,NaN,NaN,NaN,"[consent] = ""0""",NaN,NaN,NaN
gender,registration_visit,NaN,radio,Gender,"0, Female | 1, Male | 99, Undefined",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
race,registration_visit,NaN,radio,Race,"1, White or Caucasian | 2, Black or African Am...",Put other for anything not listed here (Middle...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
visitcategory,baseline_and_follow_up,NaN,radio,What type of visit is this?,"1, Baseline Visit | 2, Follow Up Visit",Please make sure this matches the event name a...,NaN,NaN,NaN,NaN,[registration_visit_arm_1][ibddiag] = '1' and ...,y,NaN,NaN
visittype,baseline_and_follow_up,NaN,radio,BV4. Visit Type,"1, Baseline Visit, within 3 months of initial ...",NaN,NaN,NaN,NaN,NaN,[visitcategory]='1',NaN,NaN,NaN


In [16]:
cb[cb['Field Type'] == 'checkbox'].head()

,Form Name,Section Header,Field Type,Field Label,"Choices, Calculations, OR Slider Labels",Field Note,Text Validation Type OR Show Slider Number,Text Validation Min,Text Validation Max,Identifier?,Branching Logic (Show field only if...),Required Field?,Custom Alignment,Matrix Group Name
Variable / Field Name,,,,,,,,,,,,,,
prior_protocol_number,registration_visit,NaN,checkbox,"If Yes, what other studies?**","5, Biorepository (P00000529) | 1, CHIMP (X06-1...",NaN,NaN,NaN,NaN,NaN,[otherstudies] = '1',NaN,NaN,NaN
sampletype,registration_visit,NaN,checkbox,What samples has the patient given?**,"1, Blood | 3, Fecal | 4, GI Biopsy | 5, Oral s...",NaN,NaN,NaN,NaN,NaN,[prior_protocol_number(5)] = '1' or [prior_pro...,NaN,NaN,NaN
historyfrom,baseline_and_follow_up,NaN,checkbox,BV10. History obtained from (choose all that a...,"1, Patient | 2, Parent or guardian | 3, Chart ...",NaN,NaN,NaN,NaN,NaN,[visitcategory]='1',NaN,LH,NaN
historyfrom2,baseline_and_follow_up,NaN,checkbox,FV11. History obtained from:,"1, Patient | 2, Parent or guardian | 3, Chart ...",NaN,NaN,NaN,NaN,NaN,[visitcategory]='2',NaN,LH,NaN
jewanctype,baseline_and_follow_up,NaN,checkbox,"If yes, indicate Jewish ancestry subtype (choo...","1, Sephardic | 2, Ashkenazi | 3, Unknown",NaN,NaN,NaN,NaN,NaN,"[jeworigin]=""1"" or [jeworiginnew]=""1""",NaN,LH,NaN


### Data

In [17]:
d = loading.load_redcap_dump(data_=data_, data_dict=cb)
d.head()

visitdate consent noconsent  \
subid redcap_event_name                                                      
1     baseline_visit_arm_1                           NaT     NaN       NaN   
      followup_visit_1_arm_1                         NaT     NaN       NaN   
      followup_visit_2_arm_1                         NaT     NaN       NaN   
      registration_visit_arm_1 2011-05-24 00:00:00+00:00     1.0       NaN   
10    baseline_visit_arm_1                           NaT     NaN       NaN   

                               noconsentoth veoibd otherstudies  \
subid redcap_event_name                                           
1     baseline_visit_arm_1              NaN    NaN          NaN   
      followup_visit_1_arm_1            NaN    NaN          NaN   
      followup_visit_2_arm_1            NaN    NaN          NaN   
      registration_visit_arm_1          NaN    NaN          1.0   
10    baseline_visit_arm_1              NaN    NaN          NaN   

                               prior_protocol_number___5  \
subid redcap_event_name                                    
1     baseline_visit_arm_1                           NaN   
      followup_visit_1_arm_1                         NaN   
      followup_visit_2_arm_1                         NaN   
      registration_visit_arm_1                       1.0   
10    baseline_visit_arm_1                           NaN   

                               prior_protocol_number___1  \
subid redcap_event_name                                    
1     baseline_visit_arm_1                           NaN   
      followup_visit_1_arm_1                         NaN   
      followup_visit_2_arm_1                         NaN   
      registration_visit_arm_1                       0.0   
10    baseline_visit_arm_1                           NaN   

                               prior_protocol_number___2  \
subid redcap_event_name                                    
1     baseline_visit_arm_1                           NaN   
      followup_visit_1_arm_1                         NaN   
      followup_visit_2_arm_1                         NaN   
      registration_visit_arm_1                       0.0   
10    baseline_visit_arm_1                           NaN   

                               prior_protocol_number___3  \
subid redcap_event_name                                    
1     baseline_visit_arm_1                           NaN   
      followup_visit_1_arm_1                         NaN   
      followup_visit_2_arm_1                         NaN   
      registration_visit_arm_1                       0.0   
10    baseline_visit_arm_1                           NaN   

                                          ...            hosp_k hosptype_k  \
subid redcap_event_name                   ...                                
1     baseline_visit_arm_1                ...               NaN        NaN   
      followup_visit_1_arm_1              ...               NaN        NaN   
      followup_visit_2_arm_1              ...               NaN        NaN   
      registration_visit_arm_1            ...               NaN        NaN   
10    baseline_visit_arm_1                ...               NaN        NaN   

                               hospibdtype_k hosp_l hosptype_l hospibdtype_l  \
subid redcap_event_name                                                        
1     baseline_visit_arm_1               NaN    NaN        NaN           NaN   
      followup_visit_1_arm_1             NaN    NaN        NaN           NaN   
      followup_visit_2_arm_1             NaN    NaN        NaN           NaN   
      registration_visit_arm_1           NaN    NaN        NaN           NaN   
10    baseline_visit_arm_1               NaN    NaN        NaN           NaN   

                               hosp_m hosptype_m hospibdtype_m  \
subid redcap_event_name                                          
1     baseline_visit_arm_1        NaN        NaN           NaN   
      followup_visit_1_arm_1      NaN   

In [20]:
choices = cb[cb['Choices, Calculations, OR Slider Labels'].notnull()]['Choices, Calculations, OR Slider Labels']

In [25]:
choices.noconsent.split('|')

['1, Not interested in Research Studies ',
 ' 2, Privacy Concerns ',
 ' 3, Time Commitment ',
 ' 4, Other, specify']

In [22]:
cb.shape

(391, 14)

In [68]:
choices.noconsent

'1, Not interested in Research Studies | 2, Privacy Concerns | 3, Time Commitment | 4, Other, specify'

In [94]:
cparser = build_choices_parser()
parsed_choices = cparser.parseString(choices.noconsent).asList()
parsed_choices

[['1', 'Not interested in Research Studies'],
 ['2', 'Privacy Concerns'],
 ['3', 'Time Commitment'],
 ['4', 'Other, specify']]

In [95]:
{i:s for i,s in parsed_choices}

{'1': 'Not interested in Research Studies',
 '2': 'Privacy Concerns',
 '3': 'Time Commitment',
 '4': 'Other, specify'}

In [121]:
choice_maps = loading.make_choice_maps(data_dict=cb)

In [124]:
choice_maps.abcessess['0']

'No'

# Subject

# Convert to HTML report and move to reports folder

In [ ]:
# rv.reprt_dir

In [ ]:
# !jupyter nbconvert --to html_toc $rv.ipnb

In [ ]:
# !mkdir -p $rv.reprt_dir
## !mv -f $rv.html $rv.files $rv.reprt_dir